In [1]:
# import libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import shapely.geometry as geom

/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# dataframe

In [3]:
# Create a DataFrame with box names and values
box_names = [chr(65 + i) for i in range(16)]
index = list(range(0, 16))
values = list(range(1, 17))

In [4]:
df = pd.DataFrame({'FIPS': box_names,'Index': index, 'Value': values})

# Sorting the DataFrame based on the 'rate' column :::: Values are needed to be sorted in ascending order
df.sort_values(by='Value', inplace=True)

# Adding a new column 'new_ID' with ID values starting from zero
df['sortedID'] = range(len(df))

# Function to calculate square coordinates for a given name
def calculate_square_coordinates(row):
    value = row['Index']
    # Assuming each smaller square has a side length of 1 unit
    x = value % 4
    y = value // 4
    # Return square coordinates as a Shapely polygon
    return geom.Polygon([(x, y), (x+1, y), (x+1, y+1), (x, y+1)])

# Calculate square coordinates for each row and create a geometry column
df['geometry'] = df.apply(calculate_square_coordinates, axis=1)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the CRS to a simple Cartesian coordinate system
gdf.crs = "EPSG:3395"  # This is a commonly used projected CRS

gdf = gdf[['FIPS','Value', 'geometry']]  

In [5]:
from adjacency_simplex import AdjacencySimplex  # Import the class

In [6]:
# Initialize the AdjacencySimplex class
adj_simplex = AdjacencySimplex(gdf, 'Value', threshold = None, filter_method = 'down')

# Filter the GeoDataFrame
filtered_df,gdf_id = adj_simplex.filter_sort_gdf()

# Calculate the adjacent countries
adj_simplex.calculate_adjacent_countries()

# Form the simplicial complex
simplex = adj_simplex.form_simplicial_complex()

In [7]:
from plot_utills import plot_simplicial_complex_gif  # Import the function

In [8]:
import io
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [9]:
gdf.head()

,FIPS,Value,geometry
0,A,1,"POLYGON ((0.000 0.000, 1.000 0.000, 1.000 1.00..."
1,B,2,"POLYGON ((1.000 0.000, 2.000 0.000, 2.000 1.00..."
2,C,3,"POLYGON ((2.000 0.000, 3.000 0.000, 3.000 1.00..."
3,D,4,"POLYGON ((3.000 0.000, 4.000 0.000, 4.000 1.00..."
4,E,5,"POLYGON ((0.000 1.000, 1.000 1.000, 1.000 2.00..."


In [10]:
filtered_df.head()

,FIPS,Value,geometry,sortedID
15,P,16,"POLYGON ((3.00000 3.00000, 4.00000 3.00000, 4....",0
14,O,15,"POLYGON ((2.00000 3.00000, 3.00000 3.00000, 3....",1
13,N,14,"POLYGON ((1.00000 3.00000, 2.00000 3.00000, 2....",2
12,M,13,"POLYGON ((0.00000 3.00000, 1.00000 3.00000, 1....",3
11,L,12,"POLYGON ((3.00000 2.00000, 4.00000 2.00000, 4....",4


In [11]:
# Initialize list_gif to store images
# list_gif = []

# Call the function
list_gif = plot_simplicial_complex_gif(dataframe=gdf_id, simplices=simplex, variable='Value')

In [12]:
list_gif[0].save('test_0.gif', save_all=True,append_images=list_gif[1:],optimize=False,duration=200,loop=0) 

In [13]:
from calculate_tda_summaries import compute_persistence

In [14]:
total_h0_points, tl, al, tml, aml = compute_persistence(simplex,filtered_df, 'Value')

## Down

In [15]:
print('Total number of H0 points:', total_h0_points)
print('Total  lifespan:', tl)
print('Average lifespan:', al)
print('Total mid lifespan:', tml)
print('Average mid lifespan:', aml)

Total number of H0 points: 16
Total  lifespan: 91.0
Average lifespan: 5.6875
Total mid lifespan: 45.5
Average mid lifespan: 2.84375


## Up

In [16]:
print('Total number of H0 points:', total_h0_points)
print('Total  lifespan:', tl)
print('Average lifespan:', al)
print('Total mid lifespan:', tml)
print('Average mid lifespan:', aml)

Total number of H0 points: 16
Total  lifespan: 151.0
Average lifespan: 9.4375
Total mid lifespan: 75.5
Average mid lifespan: 4.71875
